In [1]:
#!/usr/bin/env python
# coding: utf-8

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1691346772328_0002,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import os
import sys
import pytz
import numpy as np
import datetime
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf, lpad, translate
from datetime import datetime 
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
agora=datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc=agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
spark = SparkSession.builder.appName('T_bi_politica_credito_imob_modelo_001').getOrCreate()
sqlContext=SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
_DIA_ = '20230804'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
file='application_train_20230805.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ts_file_generation = file.split("_")[-1].replace('.csv','') + '00'
print(ts_file_generation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2023080500

In [8]:
bucket_raw = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/002_raw'.format(dia=_DIA_)
bucket_ingestion = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/001_ingestion'.format(dia=_DIA_)
bucket_control = 'bkt-datalake-grupo-01-hackathon-podacademy/{dia}/005_control'.format(dia=_DIA_)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
output_lake = 't_application_train'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/001_ingestion/application_train_20230805.csv

In [11]:
pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)
pouso.registerTempTable("pouso")
pouso.cache()
qtd=pouso.count()
print('records ingestion data:', qtd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingestion data: 307511

In [12]:
for col in pouso.columns:
    print(col + ',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SK_ID_CURR,
TARGET,
NAME_CONTRACT_TYPE,
CODE_GENDER,
FLAG_OWN_CAR,
FLAG_OWN_REALTY,
CNT_CHILDREN,
AMT_INCOME_TOTAL,
AMT_CREDIT,
AMT_ANNUITY,
AMT_GOODS_PRICE,
NAME_TYPE_SUITE,
NAME_INCOME_TYPE,
NAME_EDUCATION_TYPE,
NAME_FAMILY_STATUS,
NAME_HOUSING_TYPE,
REGION_POPULATION_RELATIVE,
DAYS_BIRTH,
DAYS_EMPLOYED,
DAYS_REGISTRATION,
DAYS_ID_PUBLISH,
OWN_CAR_AGE,
FLAG_MOBIL,
FLAG_EMP_PHONE,
FLAG_WORK_PHONE,
FLAG_CONT_MOBILE,
FLAG_PHONE,
FLAG_EMAIL,
OCCUPATION_TYPE,
CNT_FAM_MEMBERS,
REGION_RATING_CLIENT,
REGION_RATING_CLIENT_W_CITY,
WEEKDAY_APPR_PROCESS_START,
HOUR_APPR_PROCESS_START,
REG_REGION_NOT_LIVE_REGION,
REG_REGION_NOT_WORK_REGION,
LIVE_REGION_NOT_WORK_REGION,
REG_CITY_NOT_LIVE_CITY,
REG_CITY_NOT_WORK_CITY,
LIVE_CITY_NOT_WORK_CITY,
ORGANIZATION_TYPE,
EXT_SOURCE_1,
EXT_SOURCE_2,
EXT_SOURCE_3,
APARTMENTS_AVG,
BASEMENTAREA_AVG,
YEARS_BEGINEXPLUATATION_AVG,
YEARS_BUILD_AVG,
COMMONAREA_AVG,
ELEVATORS_AVG,
ENTRANCES_AVG,
FLOORSMAX_AVG,
FLOORSMIN_AVG,
LANDAREA_AVG,
LIVINGAPARTMENTS_AVG,
LIVINGA

In [13]:
lake = spark.sql(     
    """
        select
            -- padrao para todas as cargas no datalake--
            
            int(date_format(current_date,'yyyyMM')) as ref,
            int(date_format(current_date,'yyyyMM')) as ref_partition,
            
            {tsfileger} as ts_file_generation,
            {tsfileger} as ts_file_generation_partition,
            
            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,
        
            -- campos do arquivo --

            SK_ID_CURR,
            TARGET,
            NAME_CONTRACT_TYPE,
            CODE_GENDER,
            FLAG_OWN_CAR,
            FLAG_OWN_REALTY,
            CNT_CHILDREN,
            AMT_INCOME_TOTAL,
            AMT_CREDIT,
            AMT_ANNUITY,
            AMT_GOODS_PRICE,
            NAME_TYPE_SUITE,
            NAME_INCOME_TYPE,
            NAME_EDUCATION_TYPE,
            NAME_FAMILY_STATUS,
            NAME_HOUSING_TYPE,
            REGION_POPULATION_RELATIVE,
            DAYS_BIRTH,
            DAYS_EMPLOYED,
            DAYS_REGISTRATION,
            DAYS_ID_PUBLISH,
            OWN_CAR_AGE,
            FLAG_MOBIL,
            FLAG_EMP_PHONE,
            FLAG_WORK_PHONE,
            FLAG_CONT_MOBILE,
            FLAG_PHONE,
            FLAG_EMAIL,
            OCCUPATION_TYPE,
            CNT_FAM_MEMBERS,
            REGION_RATING_CLIENT,
            REGION_RATING_CLIENT_W_CITY,
            WEEKDAY_APPR_PROCESS_START,
            HOUR_APPR_PROCESS_START,
            REG_REGION_NOT_LIVE_REGION,
            REG_REGION_NOT_WORK_REGION,
            LIVE_REGION_NOT_WORK_REGION,
            REG_CITY_NOT_LIVE_CITY,
            REG_CITY_NOT_WORK_CITY,
            LIVE_CITY_NOT_WORK_CITY,
            ORGANIZATION_TYPE,
            EXT_SOURCE_1,
            EXT_SOURCE_2,
            EXT_SOURCE_3,
            APARTMENTS_AVG,
            BASEMENTAREA_AVG,
            YEARS_BEGINEXPLUATATION_AVG,
            YEARS_BUILD_AVG,
            COMMONAREA_AVG,
            ELEVATORS_AVG,
            ENTRANCES_AVG,
            FLOORSMAX_AVG,
            FLOORSMIN_AVG,
            LANDAREA_AVG,
            LIVINGAPARTMENTS_AVG,
            LIVINGAREA_AVG,
            NONLIVINGAPARTMENTS_AVG,
            NONLIVINGAREA_AVG,
            APARTMENTS_MODE,
            BASEMENTAREA_MODE,
            YEARS_BEGINEXPLUATATION_MODE,
            YEARS_BUILD_MODE,
            COMMONAREA_MODE,
            ELEVATORS_MODE,
            ENTRANCES_MODE,
            FLOORSMAX_MODE,
            FLOORSMIN_MODE,
            LANDAREA_MODE,
            LIVINGAPARTMENTS_MODE,
            LIVINGAREA_MODE,
            NONLIVINGAPARTMENTS_MODE,
            NONLIVINGAREA_MODE,
            APARTMENTS_MEDI,
            BASEMENTAREA_MEDI,
            YEARS_BEGINEXPLUATATION_MEDI,
            YEARS_BUILD_MEDI,
            COMMONAREA_MEDI,
            ELEVATORS_MEDI,
            ENTRANCES_MEDI,
            FLOORSMAX_MEDI,
            FLOORSMIN_MEDI,
            LANDAREA_MEDI,
            LIVINGAPARTMENTS_MEDI,
            LIVINGAREA_MEDI,
            NONLIVINGAPARTMENTS_MEDI,
            NONLIVINGAREA_MEDI,
            FONDKAPREMONT_MODE,
            HOUSETYPE_MODE,
            TOTALAREA_MODE,
            WALLSMATERIAL_MODE,
            EMERGENCYSTATE_MODE,
            OBS_30_CNT_SOCIAL_CIRCLE,
            DEF_30_CNT_SOCIAL_CIRCLE,
            OBS_60_CNT_SOCIAL_CIRCLE,
            DEF_60_CNT_SOCIAL_CIRCLE,
            DAYS_LAST_PHONE_CHANGE,
            FLAG_DOCUMENT_2,
            FLAG_DOCUMENT_3,
            FLAG_DOCUMENT_4,
            FLAG_DOCUMENT_5,
            FLAG_DOCUMENT_6,
            FLAG_DOCUMENT_7,
            FLAG_DOCUMENT_8,
            FLAG_DOCUMENT_9,
            FLAG_DOCUMENT_10,
            FLAG_DOCUMENT_11,
            FLAG_DOCUMENT_12,
            FLAG_DOCUMENT_13,
            FLAG_DOCUMENT_14,
            FLAG_DOCUMENT_15,
            FLAG_DOCUMENT_16,
            FLAG_DOCUMENT_17,
            FLAG_DOCUMENT_18,
            FLAG_DOCUMENT_19,
            FLAG_DOCUMENT_20,
            FLAG_DOCUMENT_21,
            AMT_REQ_CREDIT_BUREAU_HOUR,
            AMT_REQ_CREDIT_BUREAU_DAY,
            AMT_REQ_CREDIT_BUREAU_WEEK,
            AMT_REQ_CREDIT_BUREAU_MON,
            AMT_REQ_CREDIT_BUREAU_QRT,
            AMT_REQ_CREDIT_BUREAU_YEAR
            
        from
            pouso
    """.format(tsfileger=ts_file_generation,pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()  

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

307511

In [14]:
lake.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ref: integer (nullable = true)
 |-- ref_partition: integer (nullable = true)
 |-- ts_file_generation: integer (nullable = false)
 |-- ts_file_generation_partition: integer (nullable = false)
 |-- ts_proc: long (nullable = false)
 |-- ts_proc_partition: long (nullable = false)
 |-- SK_ID_CURR: string (nullable = true)
 |-- TARGET: string (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: string (nullable = true)
 |-- AMT_INCOME_TOTAL: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- AMT_GOODS_PRICE: string (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: strin

In [15]:
lake.show(4,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------+------------------+----------------------------+--------------+-----------------+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+---------

In [16]:
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+------------------+--------------+--------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
# |ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|SK_ID_CURR|TARGET|NAME_CONTRACT_TYPE|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE|NAME_TYPE_SUITE|NAME_INCOME_TYPE|NAME_EDUCATION_TYPE          |NAME_FAMILY_STATUS  |NAME_HOUSING_TYPE|REGION_POPULATION_RELATIVE|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|OWN_CAR_AGE|FLAG_MOBIL|FLAG_EMP_PHONE|FLAG_WORK_PHONE|FLAG_CONT_MOBILE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|REGION_RATING_CLIENT|REGION_RATING_CLIENT_W_CITY|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|REG_REGION_NOT_LIVE_REGION|REG_REGION_NOT_WORK_REGION|LIVE_REGION_NOT_WORK_REGION|REG_CITY_NOT_LIVE_CITY|REG_CITY_NOT_WORK_CITY|LIVE_CITY_NOT_WORK_CITY|ORGANIZATION_TYPE     |EXT_SOURCE_1       |EXT_SOURCE_2      |EXT_SOURCE_3       |APARTMENTS_AVG|BASEMENTAREA_AVG|YEARS_BEGINEXPLUATATION_AVG|YEARS_BUILD_AVG   |COMMONAREA_AVG|ELEVATORS_AVG|ENTRANCES_AVG|FLOORSMAX_AVG|FLOORSMIN_AVG|LANDAREA_AVG|LIVINGAPARTMENTS_AVG|LIVINGAREA_AVG|NONLIVINGAPARTMENTS_AVG|NONLIVINGAREA_AVG|APARTMENTS_MODE|BASEMENTAREA_MODE|YEARS_BEGINEXPLUATATION_MODE|YEARS_BUILD_MODE|COMMONAREA_MODE|ELEVATORS_MODE|ENTRANCES_MODE|FLOORSMAX_MODE|FLOORSMIN_MODE|LANDAREA_MODE|LIVINGAPARTMENTS_MODE|LIVINGAREA_MODE|NONLIVINGAPARTMENTS_MODE|NONLIVINGAREA_MODE|APARTMENTS_MEDI|BASEMENTAREA_MEDI|YEARS_BEGINEXPLUATATION_MEDI|YEARS_BUILD_MEDI|COMMONAREA_MEDI|ELEVATORS_MEDI|ENTRANCES_MEDI|FLOORSMAX_MEDI|FLOORSMIN_MEDI|LANDAREA_MEDI|LIVINGAPARTMENTS_MEDI|LIVINGAREA_MEDI|NONLIVINGAPARTMENTS_MEDI|NONLIVINGAREA_MEDI|FONDKAPREMONT_MODE|HOUSETYPE_MODE|TOTALAREA_MODE|WALLSMATERIAL_MODE|EMERGENCYSTATE_MODE|OBS_30_CNT_SOCIAL_CIRCLE|DEF_30_CNT_SOCIAL_CIRCLE|OBS_60_CNT_SOCIAL_CIRCLE|DEF_60_CNT_SOCIAL_CIRCLE|DAYS_LAST_PHONE_CHANGE|FLAG_DOCUMENT_2|FLAG_DOCUMENT_3|FLAG_DOCUMENT_4|FLAG_DOCUMENT_5|FLAG_DOCUMENT_6|FLAG_DOCUMENT_7|FLAG_DOCUMENT_8|FLAG_DOCUMENT_9|FLAG_DOCUMENT_10|FLAG_DOCUMENT_11|FLAG_DOCUMENT_12|FLAG_DOCUMENT_13|FLAG_DOCUMENT_14|FLAG_DOCUMENT_15|FLAG_DOCUMENT_16|FLAG_DOCUMENT_17|FLAG_DOCUMENT_18|FLAG_DOCUMENT_19|FLAG_DOCUMENT_20|FLAG_DOCUMENT_21|AMT_REQ_CREDIT_BUREAU_HOUR|AMT_REQ_CREDIT_BUREAU_DAY|AMT_REQ_CREDIT_BUREAU_WEEK|AMT_REQ_CREDIT_BUREAU_MON|AMT_REQ_CREDIT_BUREAU_QRT|AMT_REQ_CREDIT_BUREAU_YEAR|
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+------------------+--------------+--------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
# |202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |100002    |1     |Cash loans        |M          |N           |Y              |0           |202500.0        |406597.5  |24700.5    |351000.0       |Unaccompanied  |Working         |Secondary / secondary special|Single / not married|House / apartment|0.018801                  |-9461     |-637         |-3648.0          |-2120          |null       |1         |1             |0              |1               |1         |0         |Laborers       |1.0            |2                   |2                          |WEDNESDAY                 |10                     |0                         |0                         |0                          |0                     |0                     |0                      |Business Entity Type 3|0.08303696739132256|0.2629485927471776|0.13937578009978951|0.0247        |0.0369          |0.9722                     |0.6192            |0.0143        |0.0          |0.069        |0.0833       |0.125        |0.0369      |0.0202              |0.019         |0.0                    |0.0              |0.0252         |0.0383           |0.9722                      |0.6341          |0.0144         |0.0           |0.069         |0.0833        |0.125         |0.0377       |0.022                |0.0198         |0.0                     |0.0               |0.025          |0.0369           |0.9722                      |0.6243          |0.0144         |0.0           |0.069         |0.0833        |0.125         |0.0375       |0.0205               |0.0193         |0.0                     |0.0               |reg oper account  |block of flats|0.0149        |Stone, brick      |No                 |2.0                     |2.0                     |2.0                     |2.0                     |-1134.0               |0              |1              |0              |0              |0              |0              |0              |0              |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0.0                       |0.0                      |0.0                       |0.0                      |0.0                      |1.0                       |
# |202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |100003    |0     |Cash loans        |F          |N           |N              |0           |270000.0        |1293502.5 |35698.5    |1129500.0      |Family         |State servant   |Higher education             |Married             |House / apartment|0.003540999999999999      |-16765    |-1188        |-1186.0          |-291           |null       |1         |1             |0              |1               |1         |0         |Core staff     |2.0            |1                   |1                          |MONDAY                    |11                     |0                         |0                         |0                          |0                     |0                     |0                      |School                |0.3112673113812225 |0.6222457752555098|null               |0.0959        |0.0529          |0.9851                     |0.7959999999999999|0.0605        |0.08         |0.0345       |0.2917       |0.3333       |0.013       |0.0773              |0.0549        |0.0039                 |0.0098           |0.0924         |0.0538           |0.9851                      |0.804           |0.0497         |0.0806        |0.0345        |0.2917        |0.3333        |0.0128       |0.079                |0.0554         |0.0                     |0.0               |0.0968         |0.0529           |0.9851                      |0.7987          |0.0608         |0.08          |0.0345        |0.2917        |0.3333        |0.0132       |0.0787               |0.0558         |0.0039                  |0.01              |reg oper account  |block of flats|0.0714        |Block             |No                 |1.0                     |0.0                     |1.0                     |0.0                     |-828.0                |0              |1              |0              |0              |0              |0              |0              |0              |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0.0                       |0.0                      |0.0                       |0.0                      |0.0                      |0.0                       |
# |202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |100004    |0     |Revolving loans   |M          |Y           |Y              |0           |67500.0         |135000.0  |6750.0     |135000.0       |Unaccompanied  |Working         |Secondary / secondary special|Single / not married|House / apartment|0.010032                  |-19046    |-225         |-4260.0          |-2531          |26.0       |1         |1             |1              |1               |1         |0         |Laborers       |1.0            |2                   |2                          |MONDAY                    |9                      |0                         |0                         |0                          |0                     |0                     |0                      |Government            |null               |0.5559120833904428|0.7295666907060153 |null          |null            |null                       |null              |null          |null         |null         |null         |null         |null        |null                |null          |null                   |null             |null           |null             |null                        |null            |null           |null          |null          |null          |null          |null         |null                 |null           |null                    |null              |null           |null             |null                        |null            |null           |null          |null          |null          |null          |null         |null                 |null           |null                    |null              |null              |null          |null          |null              |null               |0.0                     |0.0                     |0.0                     |0.0                     |-815.0                |0              |0              |0              |0              |0              |0              |0              |0              |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0.0                       |0.0                      |0.0                       |0.0                      |0.0                      |0.0                       |
# |202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |100006    |0     |Cash loans        |F          |N           |Y              |0           |135000.0        |312682.5  |29686.5    |297000.0       |Unaccompanied  |Working         |Secondary / secondary special|Civil marriage      |House / apartment|0.008019                  |-19005    |-3039        |-9833.0          |-2437          |null       |1         |1             |0              |1               |0         |0         |Laborers       |2.0            |2                   |2                          |WEDNESDAY                 |17                     |0                         |0                         |0                          |0                     |0                     |0                      |Business Entity Type 3|null               |0.6504416904014653|null               |null          |null            |null                       |null              |null          |null         |null         |null         |null         |null        |null                |null          |null                   |null             |null           |null             |null                        |null            |null           |null          |null          |null          |null          |null         |null                 |null           |null                    |null              |null           |null             |null                        |null            |null           |null          |null          |null          |null          |null         |null                 |null           |null                    |null              |null              |null          |null          |null              |null               |2.0                     |0.0                     |2.0                     |0.0                     |-617.0                |0              |1              |0              |0              |0              |0              |0              |0              |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |0               |null                      |null                     |null                      |null                     |null                     |null                      |
# +------+-------------+------------------+----------------------------+--------------+-----------------+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+---------------+-----------------+----------------------------+----------------+---------------+--------------+--------------+--------------+--------------+-------------+---------------------+---------------+------------------------+------------------+------------------+--------------+--------------+------------------+-------------------+------------------------+------------------------+------------------------+------------------------+----------------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
# only showing top 4 rows
# # +------+-------------+------------------+----------------------------+----------

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)
lake.coalesce(1).write. \
partitionBy("ref_partition","ts_file_generation_partition","ts_proc_partition"). \
parquet(path_raw,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/002_raw/t_application_train

In [18]:
varDataFile = output_lake

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
controle = spark.sql(    
        """
            select
                '{tb}' as name_file,
                '{tb}' as name_file_partition,
                
                ref,
                ref_partition,
                
                ts_file_generation,
                ts_file_generation_partition,
                
                ts_proc,
                ts_proc_partition,
                
                count(*) as qtd_registros
            from 
                lake as a
            group by
                1,2,3,4,5,6,7,8
            order by
                1,2,3,4,5,6,7,8
        """.format(tb=varDataFile))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
controle.registerTempTable("controle")
controle.cache()
qtd=controle.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
controle.show(qtd,truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|name_file          |name_file_partition|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
+-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
|t_application_train|t_application_train|202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |307511       |
+-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+

In [22]:
# +-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |name_file          |name_file_partition|ref   |ref_partition|ts_file_generation|ts_file_generation_partition|ts_proc       |ts_proc_partition|qtd_registros|
# +-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+
# |t_application_train|t_application_train|202308|202308       |2023080500        |2023080500                  |20230806154927|20230806154927   |307511       |
# +-------------------+-------------------+------+-------------+------------------+----------------------------+--------------+-----------------+-------------+

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
path_control = os.path.join('s3://',bucket_control,'tb_0001_controle_procesamento_raw')
print(path_control)
controle.coalesce(1).write.\
parquet(path_control,mode='append')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-datalake-grupo-01-hackathon-podacademy/20230804/005_control/tb_0001_controle_procesamento_raw